In [1]:
users = 1 # number of clients


In [2]:
import os
import struct
import socket
import pickle
import time

import h5py
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

/Applications/anaconda3/envs/keras/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
root_path = '/Users/tej/models/cifar10_data'

In [4]:
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = "cpu"
torch.manual_seed(777)
if device =="cuda:0":
    torch.cuda.manual_seed_all(777)

In [5]:
client_order = int(input("client_order(start from 0): "))

client_order(start from 0): 0


In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Define the model
class SimpleCNN(nn.Module):
    def __init__(self):  # Corrected here
        super(SimpleCNN, self).__init__()  # Corrected here
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc = nn.Linear(32 * 16 * 16, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = x.view(-1, 32 * 16 * 16)
        x = self.fc(x)
        return x

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Divide the dataset into three equal parts for three clients
total_images = len(trainset)
split_points = [total_images // 3, 2 * total_images // 3]

client1_dataset = torch.utils.data.Subset(trainset, range(split_points[0]))
#client2_dataset = torch.utils.data.Subset(trainset, range(split_points[0], split_points[1]))
#client3_dataset = torch.utils.data.Subset(trainset, range(split_points[1], total_images))

client1_loader = torch.utils.data.DataLoader(client1_dataset, batch_size=32, shuffle=True)
#client2_loader = torch.utils.data.DataLoader(client2_dataset, batch_size=32, shuffle=True)
#client3_loader = torch.utils.data.DataLoader(client3_dataset, batch_size=32, shuffle=True)

# Print the number of clients and images for each client
print(f"Number of clients: 1")
print(f"Number of images for Client 1: {len(client1_dataset)}")
#print(f"Number of images for Client 2: {len(client2_dataset)}")
#print(f"Number of images for Client 3: {len(client3_dataset)}")

# Initialize the global model
global_model = SimpleCNN()

# Training loop
epochs = 5
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(global_model.parameters(), lr=0.01)

# Training on Client 1
for epoch in range(epochs):
    for data, target in client1_loader:
        optimizer.zero_grad()
        output = global_model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

# Training on Client 2
#for epoch in range(epochs):
#    for data, target in client2_loader:
#        optimizer.zero_grad()
#        output = global_model(data)
#        loss = criterion(output, target)
#        loss.backward()
#        optimizer.step()

# Training on Client 3
#for epoch in range(epochs):
#   for data, target in client3_loader:
#        optimizer.zero_grad()
#        output = global_model(data)
#        loss = criterion(output, target)
#        loss.backward()
#        optimizer.step()

# Evaluate the global model
class_correct = [0] * 10
class_total = [0] * 10

with torch.no_grad():
    for data, target in torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False):
        output = global_model(data)
        _, predicted = torch.max(output, 1)
        for i in range(10):
            class_total[i] += (target == i).sum().item()
            class_correct[i] += (predicted == target)[target == i].sum().item()

for i in range(10):
    accuracy = class_correct[i] / class_total[i]
    print(f"Class {i} Test Accuracy: {accuracy}")


Files already downloaded and verified
Files already downloaded and verified
Number of clients: 1
Number of images for Client 1: 16666
Class 0 Test Accuracy: 0.599
Class 1 Test Accuracy: 0.669
Class 2 Test Accuracy: 0.468
Class 3 Test Accuracy: 0.401
Class 4 Test Accuracy: 0.461
Class 5 Test Accuracy: 0.364
Class 6 Test Accuracy: 0.637
Class 7 Test Accuracy: 0.516
Class 8 Test Accuracy: 0.616
Class 9 Test Accuracy: 0.633


In [19]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

from torch.utils.data import Subset

# Total number of images in the dataset
total_images = 16666

# Number of clients
num_clients = 1

# Number of images per client
num_traindata = total_images // num_clients

# Client order (0 for the first client, 1 for the second, etc.)
client_order = 0  # Replace with the current client index

indices = list(range(total_images))

# Calculate the part of the dataset for the current client
part_tr = indices[num_traindata * client_order : num_traindata * (client_order + 1)]


In [20]:
trainset = torchvision.datasets.CIFAR10 (root=root_path, train=True, download=True, transform=transform)

trainset_sub = Subset(trainset, part_tr)

train_loader = torch.utils.data.DataLoader(trainset_sub, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10 (root=root_path, train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [21]:
x_train, y_train = next(iter(train_loader))
print(x_train.size())
print(y_train.size())

torch.Size([4, 3, 32, 32])
torch.Size([4])


In [22]:
total_batch = len(train_loader)
print(total_batch)

4167


In [23]:
# -*- coding: utf-8 -*-
"""
Created on Thu Nov  1 14:23:31 2018
@author: tshzzz
"""

import torch
import torch.nn as nn


def conv_dw_client(inplane,stride=1):
    return nn.Sequential(
        nn.Conv2d(inplane,inplane,kernel_size = 3,groups = inplane,stride=stride,padding=1),
        nn.BatchNorm2d(inplane),
        nn.ReLU()  
        )

def conv_bw(inplane,outplane,kernel_size = 3,stride=1):
    return nn.Sequential(
        nn.Conv2d(inplane,outplane,kernel_size = kernel_size,groups = 1,stride=stride,padding=1),
        nn.BatchNorm2d(outplane),
        nn.ReLU() 
        )


class MobileNet(nn.Module):
    
    def __init__(self,num_class=10):
        super(MobileNet,self).__init__()
        
        layers = []
        layers.append(conv_bw(3,32,3,1))
        layers.append(conv_dw_client(32,1))
#         layers.append(conv_dw(64,128,2))
#         layers.append(conv_dw(128,128,1))
#         layers.append(conv_dw(128,256,2))
#         layers.append(conv_dw(256,256,1))
#         layers.append(conv_dw(256,512,2))

#         for i in range(5):
#             layers.append(conv_dw(512,512,1))
#         layers.append(conv_dw(512,1024,2))
#         layers.append(conv_dw(1024,1024,1))

#         self.classifer = nn.Sequential(
#                 nn.Dropout(0.5),
#                 nn.Linear(1024,num_class)
#                 )
        self.feature = nn.Sequential(*layers)
        
        

    def forward(self,x):
        out = self.feature(x)
#         out = out.mean(3).mean(2)
#         out = out.view(-1,1024)
#         out = self.classifer(out)
        return out

In [24]:
mobilenet_client = MobileNet().to(device)
print(mobilenet_client)


MobileNet(
  (feature): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
)


In [25]:
# from torchsummary import summary

# summary(mobilenet_client, (3, 32, 32))

In [26]:
epoch = 20  # default
lr = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(mobilenet_client.parameters(), lr=lr, momentum=0.9)

In [27]:
def send_msg(sock, msg):
    # prefix each message with a 4-byte length in network byte order
    msg = pickle.dumps(msg)
    msg = struct.pack('>I', len(msg)) + msg
    sock.sendall(msg)

def recv_msg(sock):
    # read message length and unpack it into an integer
    raw_msglen = recvall(sock, 4)
    if not raw_msglen:
        return None
    msglen = struct.unpack('>I', raw_msglen)[0]
    # read the message data
    msg =  recvall(sock, msglen)
    msg = pickle.loads(msg)
    return msg

def recvall(sock, n):
    # helper function to receive n bytes or return None if EOF is hit
    data = b''
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data += packet
    return data

In [34]:
host = input("IP address: ")
port = 10081


IP address: 10.0.0.94


In [35]:
start_time = time.time()    # store start time
print("timmer start!")

timmer start!


In [36]:
s = socket.socket()
s.connect((host, port))

In [37]:
epoch = recv_msg(s)   # get epoch
msg = total_batch
send_msg(s, msg)   # send total_batch of train dataset

In [38]:
for e in range(epoch):
    client_weights = recv_msg(s)
    mobilenet_client.load_state_dict(client_weights)
    mobilenet_client.eval()
    for i, data in enumerate(tqdm(train_loader, ncols=100, desc='Epoch '+str(e+1))):
        x, label = data
        x = x.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        output = mobilenet_client(x)
        client_output = output.clone().detach().requires_grad_(True)
        msg = {
            'client_output': client_output,
            'label': label
        }
        send_msg(s, msg)
        client_grad = recv_msg(s)
        output.backward(client_grad)
        optimizer.step()
    send_msg(s, mobilenet_client.state_dict())

Epoch 1: 100%|██████████████████████████████████████████████████| 4167/4167 [22:53<00:00,  3.03it/s]


In [39]:
 end_time = time.time()  #store end time
print("WorkingTime of ",device ,": {} sec".format(end_time - start_time))

WorkingTime of  cpu : 1631.8849160671234 sec
